In [ ]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from dotenv import load_dotenv, find_dotenv
import os


In [ ]:
load_dotenv(find_dotenv())

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
base_path = os.getcwd()
vectordb_path = os.path.join(base_path, "..", "skills-library", "vectordb")


client = chromadb.PersistentClient(path=vectordb_path)

In [ ]:
# returns a nanosecond heartbeat. Useful for making sure the client remains connected.
client.heartbeat()


In [ ]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai_api_key,
    model_name="text-embedding-3-large"
)

In [ ]:
collections = client.list_collections()
collections

In [ ]:
client.delete_collection('experience_library_v1')

In [ ]:
collection1 = client.get_or_create_collection(
    "experience_library_v1", embedding_function=openai_ef
)
collection1.peek()

In [ ]:
collection = client.get_collection(
    "experience_library_v1", embedding_function=openai_ef
)

def SearchExperienceLibrary(
    conflict_description: str,
    num_ac: int,
    conflict_formation: str,
    min_distance: float,
    min_tcpa: float,
):
    """
    Search in the experience library for a similar conflict and its resolution.

    :param conflict_description: Description of the conflict in a couple of sentences, e.g., how each aircraft are positioned and headed relative to one another without numbers.
    :param num_ac: Number of aircraft involved in the conflict.
    :param conflict_formation: Formation of the conflict, options include "Head-On Formation", "T-Formation", "Parallel Formation", "Converging Formation".
    :param min_distance: Minimum distance between aircraft in conflict.
    :param min_tcpa: Minimum time to closest point of approach in seconds.
    :return: Document with similar conflict and advices on how to resolve it.
    """
    where_full = {
        "$and": [
            {"num_ac": num_ac},
            {"conflict_formation": conflict_formation},
            {"min_distance": {"$lte": min_distance}},
            {"min_tcpa": {"$lte": min_tcpa}},
        ]
    }

    where_partial_1 = {
        "$and": [
            {"num_ac": num_ac},
            {"conflict_formation": conflict_formation},
        ]
    }

    where_partial_2 = {"num_ac": num_ac}

    search_orders = [
        (where_full, "Full"),
        (where_partial_1, "Partial 1"),
        (where_partial_2, "Partial 2"),
        (None, "No filters"),
    ]

    for where, label in search_orders:
        try:
            query_results = collection.query(
                query_texts=[conflict_description], n_results=1, where=where
            )
            print(f"{label} \nquery results:", query_results)
            if query_results["documents"] and query_results["documents"][0]:
                doc = query_results["documents"][0][0]
                print(f"{label} query results:", doc)
                return doc
        except Exception as e:
            print(f"Error with {label} query:", e)

    return "No similar conflict found in the database."


# {
#     "conflict_description": "Three aircraft are in conflict. FLIGHT1 is at 15700 ft heading south, FLIGHT2 and FLIGHT3 are descending from 16557 ft and 16558 ft respectively, both heading north. The vertical separation is less than 1000 ft and horizontal separation is less than 5 nautical miles.",
#     "num_ac": 3,
#     "conflict_formation": "Parallel Formation",
#     "min_distance": 4.11,
#     "min_tcpa": 108.15,
# }

out = collection.query(query_texts=["Three aircraft are in conflict. FLIGHT1 is at 15700 ft heading south, FLIGHT2 and FLIGHT3 are descending from 16557 ft and 16558 ft respectively, both heading north. The vertical separation is less than 1000 ft and horizontal separation is less than 5 nautical miles."],)

print(out)

# SearchExperienceLibrary(conflict_description="Three aircraft are in conflict. FLIGHT1 is at 15700 ft heading south, FLIGHT2 and FLIGHT3 are descending from 16557 ft and 16558 ft respectively, both heading north. The vertical separation is less than 1000 ft and horizontal separation is less than 5 nautical miles.", num_ac=3, conflict_formation="Parallel Formation", min_distance=4.11, min_tcpa=108.15)

In [ ]:
collection.peek()

In [ ]:
collection = client.get_collection("default", embedding_function=openai_ef)
out = collection.query(query_texts=["Enable Conflict Detection and Resolution System\n  Useful commands: ASAS ON, RESO ON\n  - ASAS ON: Enable the conflict detection system.\n  - RESO ON: Enable the conflict resolution system.\n\n-"],
    n_results=1,
    where=None)
out

In [ ]:
# Get a collection object from an existing collection, by name. If it doesn't exist, create it.
collection = client.get_or_create_collection(
    name="skill_manuals", embedding_function=openai_ef, metadata={"hnsw:space": "cosine"})

In [ ]:
collection.peek()

In [ ]:
where_full = {
    "$and": [
        {"num_ac": 2},
        {"conflict_formation": 'head-on'},
    ]
}
where_partial = {"num_ac": 2}

query_results = collection.query(
    query_texts=["KL204 - KL420"], n_resu
)

query_results

In [ ]:
client.delete_collection("skill_manuals")


In [ ]:
doc = collection.get(ids=["6eea8c2c-f149-4745-9629-3ae48dce5558"])
doc

In [ ]:
txt ="""
Here is the transformed conflict report:

**Conflict Report**

**Number of Aircraft Involved:** 4 Boeing 747-400 (KL0001, KL0003, KL0005, KL0007)

**Conflict Type:** Converging Conflict

**Altitude:** All aircraft were initially at 20000 ft.

**Heading:**

* KL0001: 180°
* KL0003: 270°
* KL0005: 000°
* KL0007: 090°

**Conflict Formation:** Converging Formation

Note: The conflict formation is determined based on the initial headings and positions of the aircraft, which indicate a converging conflict scenario.
Here is the description of the effectiveness of each action taken to manage the aircraft conflict:

Initially, decreasing the altitude of one aircraft effectively reduced its proximity to another, resolving part of the conflict. This action created some separation between the aircraft, reducing the number of conflict pairs.

Subsequently, a heading adjustment for one of the remaining conflicting aircraft helped to further disperse the conflict, reducing the number of conflict pairs.

However, a command that did not alter the heading of an aircraft proved ineffective, as it failed to change the conflict situation, and the number of conflict pairs remained the same.       

A subsequent heading adjustment for another aircraft then successfully dispersed the remaining conflict, reducing the number of conflict pairs.

Finally, decreasing the altitude of another aircraft effectively resolved the remaining conflict, as it created sufficient separation between the aircraft, eliminating all conflict pairs.   

Overall, the actions that involved changes in altitude or heading adjustments were effective in resolving the conflict, as they created separation between the aircraft and dispersed the conflict.
"""
meta ={'num_ac': 4, 'conflict_formation': 'Converging Formation'}
collection.add(documents=[txt], metadatas=[meta], ids=["conflict_report_2"])

In [ ]:
client.delete_collection("skill_manuals")

In [ ]:
def QueryConflicts(input: str, num_ac: int, conflict_formation: str):
    """Query for a similar conflict and its resolution in the database.

    Parameters:
    - input: str (1 sentence short description of the conflict)
    - num_ac: int (number of aircraft in conflict)
    - conflict_formation: str (formation of the conflict: Head-On Formation, T-Formation, Parallel Formation, Converging Formation)
    Returns:
    - document that is a similar conflict and its resolution in the database
    """
    where = {
        "$and": [
            {"num_ac": num_ac},
            {"conflict_formation": conflict_formation},
        ]
    }
    try:
        query_results = collection.query(query_texts=[input], n_results=1, where=where)
        doc = query_results["documents"][0][0]
        if doc == "":
            return "No similar conflict found in the database."
        else:
            return doc
    except:
        return "No similar conflict found in the database."

print(QueryConflicts("Aircraft conflict with 4 aircraft", num_ac=4, conflict_formation='Converging Formation'))

In [ ]:
# step1: open all txt files  from specific folder
# step2: collect ids_lst, which is a filename without extension
# step3: collect documents_lst, which is a list of text content for each file
# step4: add documents to the collection


# Define the path to the directory containing the text files
folder_path = os.path.join(base_path, "..", "skills-library", "description")

# Initialize lists to hold file names without extensions and file contents
ids_lst = []
documents_lst = []
metadatas_lst = []
# Step 1: Open all txt files from specific folder
for filename in os.listdir(folder_path):
    # Check if the file is a text file
    if filename.endswith('.txt'):
        # Step 2: Collect ids_lst, which is a filename without extension
        command = os.path.splitext(filename)[0]
        ids_lst.append(command)
        metadatas_lst.append({"command": command})
        # Define the full path to the file
        file_path = os.path.join(folder_path, filename)

        # Step 3: Collect documents_lst, which is a list of text content for each file
        with open(file_path, 'r', encoding='utf-8') as file:
            documents_lst.append(file.read())

# Now ids_lst contains all filenames without their extension
# and documents_lst contains the corresponding file contents

In [ ]:
collection.upsert(ids=ids_lst, documents=documents_lst, metadatas=metadatas_lst)

In [ ]:
collection.get(include=["metadatas"])['metadatas']

In [ ]:
collection.get(ids=['ADDWPT', 'CR2E'])

In [ ]:
collection.query(
    query_texts=["Enable Conflict Detection and Resolution System\n  Useful commands: ASAS ON, RESO ON\n  - ASAS ON: Enable the conflict detection system.\n  - RESO ON: Enable the conflict resolution system.\n\n-",
                 
                 "Step 2: Set the conflict detection method\n  Useful commands: [CDMETHOD MODULE-NAME]\n  Usage details: `CDMETHOD MVP` - This command sets the conflict detection method to the'Minimum Vectoring Profile' which is an example module name.\n\n-", 
                 "Set the conflict resolution method\n  Useful commands: [RESO MODULE-NAME]\n  Usage details: `RESO EBY` - This command sets the conflict resolution method to 'Enhanced Bounded Yaw,' which is an example module name.\n\n"],
    n_results=5,
    #where={"command": "ADDWPT"}
)

In [ ]:
input = "HDG, ALT, AT, ATALT, ATDIST, ATSPD, BANK, DELWPT, DEST, DIRECT, DIST, HDG, SPD, VS"

commands_lst = list(set([item.strip() for item in input.split(",")]))
print(commands_lst)
documents_lst = collection.get(ids=commands_lst)["documents"]
documents_str = ""
for doc in documents_lst:
    documents_str += doc + "\n\n ############################## \n\n"

documents_str